In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import cv2

In [2]:
def detect_and_predict_mask(frame, facenet, model):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (100, 100), (104, 177, 123))
    facenet.setInput(blob)
    detections = facenet.forward()
    faces = []
    locs = []
    preds = []
    
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startx, starty, endx, endy) = box.astype('int')
            (startx, starty) = (max(0, startx), max(0, starty))
            (endx, endy) = (min(w-1, endx), min(h-1, endy))
            
            face = frame[starty:endy, startx:endx]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (100, 100))
            face = img_to_array(face)
            face = preprocess_input(face)
            
            faces.append(face)
            locs.append((startx, starty, endx, endy))
    
    if len(faces)>0:
        
        faces = np.array(faces, dtype='float32')
        preds = model.predict(faces, batch_size = 10)
    
    return (locs, preds)

In [3]:
prototxtpath = 'face_detector/deploy.prototxt'
wieghtspath = 'face_detector/res10_300x300_ssd_iter_140000.caffemodel'

In [4]:
facenet = cv2.dnn.readNet(prototxtpath, wieghtspath)

model = load_model('My_model')

In [5]:
source = cv2.VideoCapture(0)

In [6]:
while True:
    ret, frame = source.read()
    
    (locs, preds) = detect_and_predict_mask(frame, facenet, model)
    
    for (box, pred) in zip(locs, preds):
        (startx, starty, endx, endy) = box
        (mask, withoutmask) = pred
        
        label = 'Mask' if mask>withoutmask else 'No Mask'
        color = (0, 255, 0) if label == 'Mask' else (0, 0, 255)
        
        label = '{}: {:.2f}%'.format(label, max(mask, withoutmask)*100)
        
        cv2.putText(frame, label, (startx, starty-10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startx, starty), (endx, endy), color, 2)
        
    cv2.imshow('Frame', frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break

cv2.destroyAllWindows()
source.release()